In [1]:
import time
import requests
import pandas as pd
import datetime as dt

In [2]:
def query_pushshift(subreddit, kind = 'submission', day_window = 30, n = 300):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score']
    
    #establish base url and stem
    
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}"
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500"
    
    #instantiate empty list
    posts = []
    
    #implement for loop with time.sleep(2)
    
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(1)
        
    full = pd.concat(posts, sort = False)
    
    if kind == 'submission':
        
        #select desired columns
        
        full = full[SUBFIELDS]
        
        #drop duplicated
        
        full.drop_duplicates(inplace = True)
        
#         full = full.loc[full['is_self'] == True]
    
    #create timestamp column
    
    full['timestamp'] = full['created_utc'].map(dt.date.fromtimestamp)
    
    print('Query Complete!')
    
    return full
        

In [3]:
results = query_pushshift('TheOnion')

In [5]:
results.shape

In [8]:
results.to_csv('theonion.csv', index = False)